## Production tpr files generate
- some benchmarking needed first
 
 Protocol files are copied from https://github.com/intbio/gmx_protocols/tree/master/amber into
Protocols folder.
If others are needed modify then in the folder.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from myimports import *
display(Markdown(descr))



In [ ]:
# %%bash
#rm -rf MDProtocols

In [ ]:
#Set protocols URL
# prot_url="https://api.github.com/repos/intbio/gmx_protocols/contents/amber_gmx2020"

In [ ]:
# get_files_from_git(prot_url,'MDProtocols/')

## 7_Production run 10 000 ns - prepare files

In [ ]:
%%bash -s "$set_gmx_cmd" "$gen_parallel_tpr"
$1
#--out out --err err
mkdir -p GMX_run
gmx grompp -maxwarn 2 -f MDProtocols/7_prod.mdp -c GMX_run/6_equil.gro -p GMX_system/topol.top \
-o GMX_run/7_prod.tpr -v  -r GMX_run/6_equil.gro -n GMX_system/index.ndx -po GMX_run/mdout_7_prod.mdp -t GMX_run/6_equil.cpt

#It is useful to generate some files with different velocities for parallel runs
if [ "$2" = True ]
then
for i in `seq 0 15`; do
gmx grompp -maxwarn 2 -f MDProtocols/7_prod.mdp -c GMX_run/6_equil.gro -p GMX_system/topol.top \
-o GMX_run/7_prod_$i.tpr -v  -r GMX_run/6_equil.gro -n GMX_system/index.ndx -po GMX_run/mdout_7_prod.mdp -t GMX_run/6_equil.cpt
done
for i in `seq 0 15`; do
mkdir -p GMX_run/$i
cp GMX_run/7_prod_$i.tpr GMX_run/$i/7_prod.tpr
done
fi

## Transfer files to supercomputer

## Launch,relaunch make a chain of tasks
### NOTE: for relaunch, monitor and data pullback - better to use status.ipynb

Lomonosov-2 queues https://users.parallel.ru/wiki/pages/22-config
compute partition allows max 7 days!.
Then we will resume.
https://hpc.nih.gov/docs/job_dependencies.html

In [ ]:
#Relaunch
#!ssh $ssh_host "module load slurm gromacs/2018-gcc; cd ~/_scratch/$folder/GMX_run; sbatch -p compute -t 7-00:00:00 -N 4 -J $folder lomo2_run_cont.sh 7_prod"

## Monitor

Note: It is suggested that for monitoring and relaunch the user makes use of status.ipynb
Parts of conde below are deprecated.

### Pull back data for analysis

### Quick analysis 

Quick view

## QC analysis
### Let's look at energy, temperature pressure